In [1]:
import json
import pandas as pd
from pathlib import Path
from pprint import pprint
import datetime
import time
import numpy as np
import matplotlib as plt

import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

%matplotlib inline

/Users/home/Desenvolvimento/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
TRAINING_ROUND = 'Training_Round'
EPOCHS = 'Epochs'
SCENARIO = 'Scenario'
EMBEDDING = 'Embedding'
TAGGING_SCHEME = 'Tagging_Scheme'
CAP_DIM = 'Capitalisation_Dim'
LOWER = 'Lower'
CHAR_DIM = 'Char_LSTM_Dim'
WORD_DIM = 'Word_LSTM_Dim'
PRECISION = 'Precision'
RECALL = 'Recall'
F1_SCORE = 'F1_Score'

In [3]:
columns = [TRAINING_ROUND, EPOCHS, SCENARIO, EMBEDDING, TAGGING_SCHEME, CAP_DIM, LOWER, CHAR_DIM, WORD_DIM, PRECISION, RECALL, F1_SCORE]

In [4]:
columns

['Training_Round',
 'Epochs',
 'Scenario',
 'Embedding',
 'Tagging_Scheme',
 'Capitalisation_Dim',
 'Lower',
 'Char_LSTM_Dim',
 'Word_LSTM_Dim',
 'Precision',
 'Recall',
 'F1_Score']

In [5]:
pd.set_option('display.max_columns', 1000)
training_data_df = pd.read_csv('resultados.csv')
training_data_df 

,Training_Round,Epochs,Scenario,Embedding,Tagging_Scheme,Capitalisation_Dim,Lower,Char_LSTM_Dim,Word_LSTM_Dim,Precision,Recall,F1_Score
0,0,5,selective,Wang2Vec,iob,False,False,25,100,58.95,54.79,56.79
1,0,5,selective,Wang2Vec,iob,False,False,25,200,61.60,52.06,56.43
2,0,5,selective,Wang2Vec,iob,False,False,50,100,66.50,47.47,55.40
3,0,5,selective,Wang2Vec,iob,False,False,50,200,58.54,50.74,54.36
4,0,5,selective,Wang2Vec,iob,False,True,25,100,66.28,67.73,67.00
5,0,5,selective,Wang2Vec,iob,False,True,25,200,71.00,62.67,66.58
6,0,5,selective,Wang2Vec,iob,False,True,50,100,70.14,65.18,67.57
7,0,5,selective,Wang2Vec,iob,False,True,50,200,66.93,63.57,65.21
8,0,5,selective,Wang2Vec,iob,True,False,25,100,58.62,57.55,58.08
9,0,5,selective,Wang2Vec,iob,True,False,25,200,64.42,50.45,56.59


In [6]:
training_data_df = training_data_df[training_data_df.Lower == True]
len(training_data_df)

640

In [7]:
training_data_df.groupby([TAGGING_SCHEME]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Tagging_Scheme,,,,,,,,
iob,320.0,57.434375,13.705584,0.0,50.1900,63.180,65.6525,68.26
iobes,320.0,57.257937,13.813944,0.0,49.8075,63.205,65.7200,67.99


In [8]:
training_data_df.groupby([EMBEDDING]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Embedding,,,,,,,,
Wang2Vec,160.0,61.226500,16.251778,0.00,65.6750,66.390,66.930,68.26
FastText,160.0,60.345562,15.044598,0.00,64.0800,65.015,65.705,67.05
Glove,160.0,59.373563,12.268520,1.01,61.5675,62.610,63.255,64.89
Word2Vec,160.0,48.439000,3.254962,16.37,47.4900,48.950,49.945,52.42


In [9]:
training_data_df.groupby([CAP_DIM]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Capitalisation_Dim,,,,,,,,
True,320.0,58.725250,11.244064,0.0,50.1750,63.410,65.9750,68.26
False,320.0,55.967063,15.761947,0.0,49.8175,62.985,65.5425,68.07


In [10]:
training_data_df.groupby([CHAR_DIM]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Char_LSTM_Dim,,,,,,,,
25,320.0,57.639781,13.521744,0.0,50.0375,63.175,65.7050,68.26
50,320.0,57.052531,13.988322,0.0,49.8575,63.225,65.6375,68.07


In [11]:
training_data_df.groupby([WORD_DIM]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Word_LSTM_Dim,,,,,,,,
100,320.0,58.037719,12.660791,0.00,50.175,63.280,65.8775,68.26
200,320.0,56.654594,14.745458,0.55,49.645,63.145,65.5925,68.07


In [12]:
training_data_df.groupby([EMBEDDING, TAGGING_SCHEME, CAP_DIM, WORD_DIM, CHAR_DIM]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

count  \
Embedding Tagging_Scheme Capitalisation_Dim Word_LSTM_Dim Char_LSTM_Dim          
Wang2Vec  iob            False              100           50              10.0   
          iobes          True               200           25              10.0   
          iob            True               100           50              10.0   
          iobes          True               100           25              10.0   
                                                          50              10.0   
                                            200           50              10.0   
FastText  iob            True               100           25              10.0   
                         False              100           25              10.0   
          iobes          True               200           25              10.0   
          iob            False              200           50              10.0   
          iobes          False              100           25              10.0   
Wang2Vec  iob            True               100           25              10.0   
FastText  iob            True               200           50              10.0   
          iobes          True               100           25              10.0   
Glove     iob            True               100           50              10.0   
Wang2Vec  iob            True               200           50              10.0   
Glove     iobes          True               100           25              10.0   
          iob            False              100           50              10.0   
                                            200           25              10.0   
                         True               200           25              10.0   
          iobes          True               200           50              10.0   
FastText  iob            True               100           50              10.0   
Wang2Vec  iob            True               200           25              10.0   
Glove     iobes          True               100           50              10.0   
Wang2Vec  iob            False              200           25              10.0   
FastText  iob            True               200           25              10.0   
Wang2Vec  iobes          False              200           50              10.0   
FastText  iobes          False              100           50              10.0   
Wang2Vec  iobes          False              100           25              10.0   
Glove     iobes          False              200           50              10.0   
...                                                                        ...   
                                            100           50              10.0   
FastText  iob            False              100           50              10.0   
Glove     iob            True               100           25              10.0   
          iobes          False              200           25              10.0   
          iob            False              200           50              10.0   
          iobes          True               200           25              10.0   
FastText  iobes          True               200           50              10.0   
          iob            False              200           25              10.0   
Wang2Vec  iobes          False              100           50              10.0   
Glove     iobes          False              100           25              10.0   
Wang2Vec  iob            False              100           25              10.0   
FastText  iobes          True               100           50              10.0   
                         False              200           50              10.0   
Word2Vec  iob            False              100           25              10.0   
          iobes          True               100           50              10.0   
          iob            False              200           50              10.0   
          iobes          False              200        

In [13]:
training_data_df.groupby([TRAINING_ROUND]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Training_Round,,,,,,,,
8,64.0,59.151563,9.073109,22.55,50.5700,63.190,65.4825,67.73
1,64.0,58.443906,12.542331,0.00,50.2350,63.265,66.0225,67.99
2,64.0,58.219688,13.621895,0.00,51.4425,64.015,66.0125,68.07
0,64.0,57.934844,13.036666,1.88,50.4950,63.160,65.7625,67.57
6,64.0,57.797500,13.314416,0.00,49.8125,63.240,65.5600,67.74
9,64.0,57.722500,12.003204,1.39,49.9450,63.080,65.7500,67.87
3,64.0,57.347812,14.015053,0.95,49.7275,63.530,65.8175,67.79
5,64.0,56.333125,15.403058,0.55,49.2400,63.280,66.0825,67.86
7,64.0,55.876719,15.756273,1.61,48.9275,62.400,65.4600,67.53


In [14]:
import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

In [15]:
rp.summary_cont(training_data_df[F1_SCORE])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,F1_Score,640.0,57.346156,13.749383,0.543492,56.27891,58.413403


In [16]:
rp.summary_cont(training_data_df.groupby(EMBEDDING))[F1_SCORE]

,N,Mean,SD,SE,95% Conf.,Interval
Embedding,,,,,,
FastText,160,60.345562,15.044598,1.189380,58.014378,62.676747
Glove,160,59.373563,12.268520,0.969912,57.472536,61.274589
Wang2Vec,160,61.226500,16.251778,1.284816,58.708261,63.744739
Word2Vec,160,48.439000,3.254962,0.257327,47.934638,48.943362


In [17]:
print(F1_SCORE, EMBEDDING, TAGGING_SCHEME, CAP_DIM, CHAR_DIM, WORD_DIM)

F1_Score Embedding Tagging_Scheme Capitalisation_Dim Char_LSTM_Dim Word_LSTM_Dim


In [28]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('F1_Score ~ C(Embedding)*C(Tagging_Scheme)*C(Capitalisation_Dim)*C(Char_LSTM_Dim)*C(Word_LSTM_Dim)', training_data_df).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .20f}")

Overall model F( 63, 576) =  2.720, p =  0.00000000044093326806


In [19]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               F1_Score   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     2.720
Date:                Mon, 08 Apr 2019   Prob (F-statistic):           4.41e-10
Time:                        21:57:03   Log-Likelihood:                -2501.7
No. Observations:                 640   AIC:                             5131.
Df Residuals:                     576   BIC:                             5417.
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================================================================================
                                                                                                                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                                           65.2590      4.020     16.232      0.000      57.363      73.155
C(Embedding)[T.Glove]                                                                                                               -7.4640      5.686     -1.313      0.190     -18.631       3.703
C(Embedding)[T.Wang2Vec]                                                                                                           -11.7700      5.686     -2.070      0.039     -22.937      -0.603
C(Embedding)[T.Word2Vec]                                                                                                           -15.6220      5.686     -2.748      0.006     -26.789      -4.455
C(Tagging_Scheme)[T.iobes]                                                                                                          -0.8910      5.686     -0.157      0.876     -12.058      10.276
C(Capitalisation_Dim)[T.True]                                                                                                        0.5770      5.686      0.101      0.919     -10.590      11.744
C(Char_LSTM_Dim)[T.50]                                                                                                              -8.2240      5.686     -1.446      0.149     -19.391       2.943
C(Word_LSTM_Dim)[T.200]                                                                                                             -9.1300      5.686     -1.606      0.109     -20.297       2.037
C(Embedding)[T.Glove]:C(Tagging_Scheme)[T.iobes]                                                                                    -1.4700      8.041     -0.183      0.855     -17.263      14.323
C(Embedding)[T.Wang2Vec]:C(Tagging_Scheme)[T.iobes]                                                                                  7.1930      8.041      0.895      0.371      -8.600      22.986
C(Embedding)[T.Word2Vec]:C(Tagging_Scheme)[T.iobes]                                                                                 -0.8080      8.041     -0.100      0.920     -16.601      14.985
C(Embedding)[T.Glove]:C(Capitalisation_Dim)[T.True]                                                                                 -1.3420      8.041     -0.167      0.868     -17.135      14.451
C(Embedding)[T.Wang2Vec]:C(Capitalisation_Dim)[T.True]                                                                          

In [20]:
res = sm.stats.anova_lm(model, typ= 2)

In [21]:
res

,sum_sq,df,F,PR(>F)
C(Embedding),17200.210246,3.0,35.471876,4.772870e-21
C(Tagging_Scheme),4.980831,1.0,0.030816,8.607137e-01
C(Capitalisation_Dim),1217.215726,1.0,7.530767,6.254498e-03
C(Char_LSTM_Dim),55.178010,1.0,0.341380,5.592627e-01
C(Word_LSTM_Dim),306.085563,1.0,1.893714,1.693183e-01
C(Embedding):C(Tagging_Scheme),676.420998,3.0,1.394978,2.433365e-01
C(Embedding):C(Capitalisation_Dim),1388.794186,3.0,2.864101,3.613673e-02
C(Tagging_Scheme):C(Capitalisation_Dim),5.802631,1.0,0.035900,8.497885e-01
C(Embedding):C(Char_LSTM_Dim),657.834294,3.0,1.356647,2.551479e-01
C(Tagging_Scheme):C(Char_LSTM_Dim),84.419303,1.0,0.522292,4.701589e-01


In [22]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(Embedding),17200.210246,5733.403415,3.0,35.471876,4.772870e-21,1.423857e-01,0.138187
C(Tagging_Scheme),4.980831,4.980831,1.0,0.030816,8.607137e-01,4.123201e-05,-0.001295
C(Capitalisation_Dim),1217.215726,1217.215726,1.0,7.530767,6.254498e-03,1.007628e-02,0.008727
C(Char_LSTM_Dim),55.178010,55.178010,1.0,0.341380,5.592627e-01,4.567713e-04,-0.000880
C(Word_LSTM_Dim),306.085563,306.085563,1.0,1.893714,1.693183e-01,2.533819e-03,0.001194
C(Embedding):C(Tagging_Scheme),676.420998,225.473666,3.0,1.394978,2.433365e-01,5.599507e-03,0.001583
C(Embedding):C(Capitalisation_Dim),1388.794186,462.931395,3.0,2.864101,3.613673e-02,1.149663e-02,0.007473
C(Tagging_Scheme):C(Capitalisation_Dim),5.802631,5.802631,1.0,0.035900,8.497885e-01,4.803498e-05,-0.001288
C(Embedding):C(Char_LSTM_Dim),657.834294,219.278098,3.0,1.356647,2.551479e-01,5.445644e-03,0.001430
C(Tagging_Scheme):C(Char_LSTM_Dim),84.419303,84.419303,1.0,0.522292,4.701589e-01,6.988347e-04,-0.000638


In [23]:
mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[EMBEDDING])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
 group1   group2  meandiff  lower    upper  reject
--------------------------------------------------
FastText  Glove    -0.972  -4.6476   2.7036 False 
FastText Wang2Vec  0.8809  -2.7947   4.5566 False 
FastText Word2Vec -11.9066 -15.5822 -8.2309  True 
 Glove   Wang2Vec  1.8529  -1.8227   5.5286 False 
 Glove   Word2Vec -10.9346 -14.6102 -7.2589  True 
Wang2Vec Word2Vec -12.7875 -16.4631 -9.1119  True 
--------------------------------------------------


In [24]:
mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[CAP_DIM])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff lower  upper  reject
-------------------------------------------
False   True   2.7582  0.6328 4.8836  True 
-------------------------------------------


In [25]:
mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[TAGGING_SCHEME])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
 iob   iobes  -0.1764  -2.3126 1.9597 False 
--------------------------------------------


In [26]:
mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[CHAR_DIM])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  25     50   -0.5872  -2.7229 1.5484 False 
--------------------------------------------


In [27]:
mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[WORD_DIM])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
 100    200   -1.3831  -3.5166 0.7503 False 
--------------------------------------------
